### Imports

In [41]:
import pandas as pd

### Helpers

In [42]:
def upsample_to_hourly(df, date_col):
    """Convert daily/monthly to hourly using forward fill."""
    return (
        df.set_index(date_col)
          .resample("h")
          .ffill()
          .reindex(hourly_index)
          .ffill()
          .reset_index()
          .rename(columns={"index": "Date (UTC)"})
    )

### Read Data

In [43]:
xls = pd.ExcelFile("Case Study Data .xlsx")
df_prices = pd.read_excel(xls, sheet_name="Day-ahead Prices")
df_genmix = pd.read_excel(xls, sheet_name="Generation Mix")
df_load = pd.read_excel(xls, sheet_name="Load")
df_import = pd.read_excel(xls, sheet_name="Import_Export")
df_solarwind = pd.read_excel(xls, sheet_name="Installed Solar Wind Capacity")
df_natgas = pd.read_excel(xls, sheet_name="Natural Gas Prices")
df_co2 = pd.read_excel(xls, sheet_name="CO2 Prices")
df_coal = pd.read_excel(xls, sheet_name="Coal Prices")

### Cleanup Time Formats

In [44]:
df_prices["Date (UTC)"] = pd.to_datetime(df_prices["Date (UTC)"])
df_genmix["Date (UTC)"] = pd.to_datetime(df_genmix["Date (UTC)"])
df_load["Date (UTC)"] = pd.to_datetime(df_load["Date (UTC)"])
df_import["Date (UTC)"] = pd.to_datetime(df_import["Date (UTC)"])
df_solarwind["Month"] = pd.to_datetime(df_solarwind["Month"])
df_natgas["Date"] = pd.to_datetime(df_natgas["Date"])
df_co2["Date"] = pd.to_datetime(df_co2["Date"], dayfirst=True)
df_coal["Date"] = pd.to_datetime(df_coal["Date"])

In [45]:
hourly_index = pd.date_range(
    start=df_prices["Date (UTC)"].min(),
    end=df_prices["Date (UTC)"].max(),
    freq="h")

df_solarwind_hr = upsample_to_hourly(df_solarwind, "Month")
df_natgas_hr = upsample_to_hourly(df_natgas, "Date")
df_co2_hr = upsample_to_hourly(df_co2, "Date")
df_coal_hr = upsample_to_hourly(df_coal, "Date")

### Merge

In [46]:
df_all = df_prices.merge(df_genmix, on="Date (UTC)", how="left")
df_all = df_all.merge(df_load, on="Date (UTC)", how="left")
df_all = df_all.merge(df_import, on="Date (UTC)", how="left")
df_all = df_all.merge(df_solarwind_hr, on="Date (UTC)", how="left")
df_all = df_all.merge(df_natgas_hr, on="Date (UTC)", how="left")
df_all = df_all.merge(df_co2_hr, on="Date (UTC)", how="left")
df_all = df_all.merge(df_coal_hr, on="Date (UTC)", how="left")

### Clean

In [47]:
df_all.rename(columns={"Amount": "Export/Import",
"Price EUR/MWh": "Natural Gas EUR/MWh",
"Price per ton CO2": "CO2 EUR/ton",
"Price EUR per T": "Coal EUR/ton",
"Price PLN/MWh": "Electricity PLN/MWh"}, inplace=True)

mask_export = df_all["Export / Import (Daily)"].str.strip().str.lower() == "export"
df_all.loc[mask_export, "Export/Import"] *= -1
df_all.drop(columns=["Export / Import (Daily)"], inplace=True)

df_all.drop(columns=["Electricity PLN/MWh"], inplace=True)
df_all["CO2 EUR/ton"].bfill(inplace=True)

/var/folders/8w/zmlxvyj52l7_fdlcpjyc4rd40000gn/T/ipykernel_74272/2364376034.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_all["CO2 EUR/ton"].bfill(inplace=True)


### Check Before Saving

In [50]:
print(df_all.describe().T.round(1))

                            count                           mean  \
Date (UTC)                  21456  2023-03-23 23:29:59.997779712   
Day-ahead Price EUR       21456.0                     129.036827   
Biomass                   21456.0                     209.987463   
Brown Coal/Lignite        21456.0                     4118.43997   
Coal-derived Gas          21456.0                      58.182933   
Natural Gas               21456.0                    1288.008296   
Hard Coal                 21456.0                     7827.04409   
Oil                       21456.0                     280.803971   
Hydro Pumped Storage      21456.0                     169.838367   
Hydro Run of River        21456.0                     200.850298   
Hydro Water Reservoir     21456.0                      14.600671   
Solar                     21456.0                    1390.655108   
Wind                      21456.0                    2449.609293   
Day-ahead Load Forecast   21456.0               

### Save

In [24]:
df_all.to_csv("clean_data.csv", index=False)